In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Data Files/hdb2017onwards.csv")

In [ ]:
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [ ]:
new_data = data.drop(["flat_type","block","street_name","remaining_lease"],axis=1)
new_data.head()

,month,town,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2017-01,ANG MO KIO,10 TO 12,44.0,Improved,1979,232000.0
1,2017-01,ANG MO KIO,01 TO 03,67.0,New Generation,1978,250000.0
2,2017-01,ANG MO KIO,01 TO 03,67.0,New Generation,1980,262000.0
3,2017-01,ANG MO KIO,04 TO 06,68.0,New Generation,1980,265000.0
4,2017-01,ANG MO KIO,01 TO 03,67.0,New Generation,1980,265000.0


In [ ]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120363 entries, 0 to 120362
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                120363 non-null  object 
 1   town                 120363 non-null  object 
 2   storey_range         120363 non-null  object 
 3   floor_area_sqm       120363 non-null  float64
 4   flat_model           120363 non-null  object 
 5   lease_commence_date  120363 non-null  int64  
 6   resale_price         120363 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.4+ MB


In [ ]:
## preprocess: town, storey_range, flat_model : dummy

In [ ]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import math
encoder = OrdinalEncoder()
columns = new_data.columns.tolist()
columns.remove("resale_price")
dummy_data = pd.DataFrame()
for col in columns:
  arr = encoder.fit_transform(new_data[[col]])
  dummy_data[col] = pd.Series(arr.reshape(1, arr.shape[0])[0])

In [ ]:
dummy_data

,month,town,storey_range,floor_area_sqm,flat_model,lease_commence_date
0,0.0,0.0,3.0,10.0,4.0,13.0
1,0.0,0.0,0.0,35.0,11.0,12.0
2,0.0,0.0,0.0,35.0,11.0,14.0
3,0.0,0.0,1.0,36.0,11.0,14.0
4,0.0,0.0,0.0,35.0,11.0,14.0
...,...,...,...,...,...,...
120358,61.0,25.0,2.0,96.0,4.0,22.0
120359,61.0,25.0,2.0,133.0,2.0,26.0
120360,61.0,25.0,3.0,115.0,6.0,21.0
120361,61.0,25.0,1.0,140.0,10.0,21.0


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = dummy_data.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(dummy_data.values, i) for i in range(len(dummy_data.columns))]
print(vif_data)

               feature       VIF
0                month  3.944905
1                 town  4.095133
2         storey_range  2.449085
3       floor_area_sqm  5.662469
4           flat_model  3.511843
5  lease_commence_date  6.565069


In [ ]:
trim_data = data
trim_data["street_name"] = "Blk " + data["block"] + " " + data["street_name"]
trim_data = trim_data.drop(["block", "remaining_lease", "flat_type"], axis=1)
trim_data = trim_data.drop(["street_name"], axis=1)
trim_data.head()

,month,town,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2017-01,ANG MO KIO,10 TO 12,44.0,Improved,1979,232000.0
1,2017-01,ANG MO KIO,01 TO 03,67.0,New Generation,1978,250000.0
2,2017-01,ANG MO KIO,01 TO 03,67.0,New Generation,1980,262000.0
3,2017-01,ANG MO KIO,04 TO 06,68.0,New Generation,1980,265000.0
4,2017-01,ANG MO KIO,01 TO 03,67.0,New Generation,1980,265000.0


In [ ]:
# NEW, extract headers and coefficients directly
# Remove handle_unknown
original_headers = trim_data.columns.values
month = trim_data['month'].unique()
storey_range = trim_data['storey_range'].unique()
town = trim_data['town'].unique()
flat_model = trim_data['flat_model'].unique()


import json
with open('month.json', 'w') as outfile:
    json.dump(month.tolist(), outfile)


with open('storey_range.json', 'w') as outfile:
    json.dump(storey_range.tolist(), outfile)


with open('town.json', 'w') as outfile:
    json.dump(town.tolist(), outfile)


with open('flat_model.json', 'w') as outfile:
    json.dump(flat_model.tolist(), outfile)

size = original_headers.size - 2 + town.size + flat_model.size - 1
full_headers = np.zeros(size, dtype=object)
full_headers_i = 0
for i in range(len(original_headers)):
  if original_headers[i] == 'town':
    for i in range(len(town)):   
      full_headers[full_headers_i] = town[i]
      full_headers_i += 1
  elif original_headers[i] == 'flat_model':
    for i in range(len(flat_model)):   
      full_headers[full_headers_i] = flat_model[i]
      full_headers_i += 1
  elif original_headers[i] == 'resale_price':
    continue
  else:
    full_headers[full_headers_i] = original_headers[i]
    full_headers_i += 1
print(full_headers)


['month' 'ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH'
 'BUKIT PANJANG' 'BUKIT TIMAH' 'CENTRAL AREA' 'CHOA CHU KANG' 'CLEMENTI'
 'GEYLANG' 'HOUGANG' 'JURONG EAST' 'JURONG WEST' 'KALLANG/WHAMPOA'
 'MARINE PARADE' 'PASIR RIS' 'PUNGGOL' 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG'
 'SERANGOON' 'TAMPINES' 'TOA PAYOH' 'WOODLANDS' 'YISHUN' 'storey_range'
 'floor_area_sqm' 'Improved' 'New Generation' 'DBSS' 'Standard'
 'Apartment' 'Simplified' 'Model A' 'Premium Apartment' 'Adjoined flat'
 'Model A-Maisonette' 'Maisonette' 'Type S1' 'Type S2' 'Model A2'
 'Terrace' 'Improved-Maisonette' 'Premium Maisonette' 'Multi Generation'
 'Premium Apartment Loft' '2-room' 'lease_commence_date']


In [ ]:
pipeline = ColumnTransformer([
("o", OrdinalEncoder(), [trim_data.columns.get_loc("month"),trim_data.columns.get_loc("storey_range")]),
("n", OneHotEncoder(), [trim_data.columns.get_loc("town"), trim_data.columns.get_loc("flat_model")]),
], remainder='passthrough')
X = pipeline.fit_transform(trim_data.drop(["resale_price"], axis=1).values.tolist())
y = trim_data["resale_price"].values.tolist()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
rr = Ridge(alpha = 10, random_state=0).fit(X_train, y_train)
print("Intercept: ", rr.intercept_)
print("Coefficient: ", rr.coef_)
print("R-sq of model: ", rr.score(X_train, y_train))

Intercept:  -10148135.629761089
Coefficient:  [ 9.26916248e+02  1.42089037e+04  3.46200837e+04  2.03271659e+04
  1.37070033e+05 -5.22017655e+04  1.42474327e+05 -1.03994270e+05
  6.25816421e+04  1.21480457e+05 -1.39471666e+05  7.24798479e+04
  7.41913690e+04 -5.10744211e+04 -3.91532727e+04 -9.70920490e+04
  9.98577522e+04  8.59457430e+04 -7.13413553e+04 -9.52252948e+04
  1.64151354e+05 -1.53479447e+05 -1.14331216e+05  3.77562587e+04
 -1.80295421e+04  8.45483312e+04 -1.26700770e+05 -7.53892953e+04
 -1.00093389e+03  6.35768992e+03 -2.10673047e+04  9.68288330e+04
 -3.75289985e+04  2.31796612e+03  2.34759181e+04 -4.58126786e+04
  1.42077774e+04 -4.12453626e+04  4.81058482e+03 -1.59136240e+04
 -3.33252876e+04  7.18625278e+03  1.02198098e+02 -2.19488115e+04
 -3.30054972e+04  2.21298439e+04  4.05089787e+04  3.29224558e+04
  4.44949522e+03  5.10022834e+03]
R-sq of model:  0.8480707524209137


In [ ]:
predicted = rr.predict(X_test[:])
rr.score(X_test, y_test)

0.8505222244415434

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 0.5, 1, 2, 5, 10, 100], cv = 10, fit_intercept = True).fit(X, y)

In [ ]:
print(clf.get_params())
print("Intercept: ", clf.intercept_)
print("Coefficient: ", clf.coef_)
print("R-sq of model: ", clf.best_score_)

{'alpha_per_target': False, 'alphas': array([1.e-03, 1.e-02, 1.e-01, 5.e-01, 1.e+00, 2.e+00, 5.e+00, 1.e+01,
       1.e+02]), 'cv': 10, 'fit_intercept': True, 'gcv_mode': None, 'normalize': 'deprecated', 'scoring': None, 'store_cv_values': False}
Intercept:  -10192143.54503731
Coefficient:  [ 9.27553749e+02  1.41449559e+04  3.49392953e+04  2.05563010e+04
  1.37173440e+05 -5.23573154e+04  1.43216487e+05 -1.04936820e+05
  6.14099687e+04  1.22688186e+05 -1.40322311e+05  7.30320168e+04
  7.57388549e+04 -5.15994025e+04 -3.88337854e+04 -9.78859218e+04
  1.00451976e+05  8.66508482e+04 -7.26185584e+04 -9.57161066e+04
  1.64936190e+05 -1.54299259e+05 -1.14536505e+05  3.87684527e+04
 -1.81803543e+04  8.43840016e+04 -1.27348612e+05 -7.53110677e+04
 -8.69409698e+02  7.56710869e+03 -2.11935570e+04  9.65931161e+04
 -3.81401083e+04  2.19273754e+03  2.30316442e+04 -4.63907898e+04
  1.53869506e+04 -4.19433112e+04  4.91547221e+03 -1.63585499e+04
 -3.39197937e+04  8.27007731e+03  4.23547809e+01 -2.201276